# Resolução de Exercício de Avaliação 

Este notebook contém a resolução de quatro questões usando métodos numéricos em Python, abordando os seguintes tópicos:

1. Regra de Simpson composta para estimar a distância que o corredor cobriu durante um dado tempo.
2. Regra dos Trapézios Generalizada aplicando a regra para estimar a pressão total sobre a parede.
3. Uso do método de Gauss-Seidel para resolver um sistema de equações que representa o balanceamento de carga
em diferentes nós de uma rede.
4. Calcule de um número aproximado de computadores de cada tipo produzidos por dia.

In [237]:
from tabulate import tabulate
from math import sqrt
import numpy as np
from numpy import array, zeros, diag, diagflat, dot


# Definindo métodos

## Método dos trapézios

In [216]:
# Definindo o método dos trapézios
def trapezios(f, h, n, a, b):
    results = []
    i = a
    j = 0
    while i <= b:
        y = round(f(i),4)
        if j == 0 or j == n:
            results.append([j, i, y]) # Guarda os valores de y
        else:
            results.append([j, i, 2*y])
        i+=h
        j+=1
    
    yn = []
    for row in results:
        y_result = row[2]
        yn.append(y_result)

    result = (h/2) * sum(yn)
    return result, results

## Método de Simpson Composto

In [125]:
# Definindo o método de Simpson Composto
# Lembrando que o intervalo tem que se par, caso contrário, não funcionará
def simpson(f, h, n, a, b):
    if n%2 != 0:
        return("Erro: n não pode ser ímpar")
    else:
        results = []
        i = a #Definindo o começo do intervalo a se calcular os valores de cada y
        j = 0 #Definindo contador
        while i <= b: #While para criar a lista com os resultados desejados
            y = round(f(i),6)
            if j == 0 or j == n:
                results.append(j, i, y) #Guarda os valores de y
            elif j%2 == 0:
                results.append(j, i, 2*y) # Guarda os valores de y
            else:
                results.append(j, i, 4*y)
            i+=h
            j+=1
            yn = []
        for row in results:
            y_result = row[2]
            yn.append(y_result)

    result = (h/3) * sum(yn)
    return result, results

In [222]:
#Check if the matrix is diagonal dominant
def check_diaDom(A):
    r = len(A)
    # Always good idea to see if the code is working the way it should be
    # so, I have included print option to check throughout the process
    diagnl = 0
    non_diagnl = 0

    for i in range(r):
        for j in range(r):
            # print(f'Loop i = {i}, j = {j}\n')
            # print(A[i, j])
            if i ==j:
             diagnl+=abs(A[i] [j])
            elif i !=j: #here you can simply use else rather than else if
             non_diagnl+=abs(A[i] [j])
    
    # print('Sum of diagonal elements\n', diagnl)
    # print('Sum of non-diagonal elements\n', non_diagnl)

    if diagnl >=non_diagnl:
       verdict = True
       #print('Diagonal dominant Matrix')
    else:
       #print('Non-diagonal dominant Matrix')
       verdict = False
    return verdict

def GS_method(A, Y, X, tol=0.001, max_iterations=1000):
    n = len(A)
    for iteration in range(max_iterations):
        X_old = X.copy()
        for j in range(n):
            summ_val = Y[j]
            for i in range(n):
                if j != i:
                    summ_val -= A[j][i] * X[i]
            X[j] = summ_val / A[j][j]
        # Compute the maximum change in X
        diff = max(abs(X[j] - X_old[j]) for j in range(n))
        if diff < tol:
            print(f"Converged after {iteration+1} iterations.")
            break
    else:
        print("Maximum iterations reached without convergence.")
    return X

# Resolução de Questões

## Teste com exemplo dos slides

In [172]:
def ft(x):
    return sqrt(1+x**3)

In [173]:
# Definir o número de intervalos a ser utilizados
n = 6

# Definir a h do intervalo a ser calculado
a = 1
b = 2

h = (b-a)/n

In [177]:
resposta, vet = trapezios(ft,h,n,a,b)

In [178]:
print (resposta)

2.1320499999999996


In [179]:
print(tabulate(vet, headers=["n", "xn", "y"], tablefmt="fancy_grid"))

╒═════╤═════════╤════════╕
│   n │      xn │      y │
╞═════╪═════════╪════════╡
│   0 │ 1       │ 1.4142 │
├─────┼─────────┼────────┤
│   1 │ 1.16667 │ 3.2174 │
├─────┼─────────┼────────┤
│   2 │ 1.33333 │ 3.6718 │
├─────┼─────────┼────────┤
│   3 │ 1.5     │ 4.1834 │
├─────┼─────────┼────────┤
│   4 │ 1.66667 │ 4.7454 │
├─────┼─────────┼────────┤
│   5 │ 1.83333 │ 5.3524 │
├─────┼─────────┼────────┤
│   6 │ 2       │ 3      │
╘═════╧═════════╧════════╛


## Questão 1

In [204]:
# Dados da tabela (t e v(t))
t = np.array([0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])
v = np.array([0, 4.67, 7.34, 8.86, 9.73, 10.22, 10.51, 10.67, 10.76, 10.81, 10.81])


In [205]:
# Adaptando a função previamente criada para o uso na questão
# Pois como já são fornecidos os x e f(x) para cada x, a função antiga só funciona quando definimos a f(x)

def simpson_composta(t, v):
    n = len(t) - 1  # Número de subintervalos
    h = (t[-1] - t[0]) / n  # Passo entre os intervalos
    
    # Aplicando a regra de Simpson composta
    soma = v[0] + v[-1]  # Primeiro e último termos
    for i in range(1, n, 2):
        soma += 4 * v[i]  # Coeficiente 4 para ímpares
    for i in range(2, n-1, 2):
        soma += 2 * v[i]  # Coeficiente 2 para pares
    
    resultado = (h / 3) * soma
    return resultado

In [206]:
distancia = simpson_composta(t, v)
print(f"Distância percorrida em 5 segundos: {distancia:.4f} metros")

Distância percorrida em 5 segundos: 44.7350 metros


## Questão 2

In [208]:
# Definindo a função a ser calculada a integral
def f(x):
    return 5*(x**2) + 3*x + 2

In [218]:
# Definir o número de intervalos a ser utilizados
n = 5

# Definir a h do intervalo a ser calculado
a = 0
b = 10

h = (b-a)/n

In [219]:
resposta, vet = trapezios(f,h,n,a,b)

In [220]:
print(resposta)

1870.0


In [221]:
print(tabulate(vet, headers=["n", "xn", "y"], tablefmt="fancy_grid"))

╒═════╤══════╤═════╕
│   n │   xn │   y │
╞═════╪══════╪═════╡
│   0 │    0 │   2 │
├─────┼──────┼─────┤
│   1 │    2 │  56 │
├─────┼──────┼─────┤
│   2 │    4 │ 188 │
├─────┼──────┼─────┤
│   3 │    6 │ 400 │
├─────┼──────┼─────┤
│   4 │    8 │ 692 │
├─────┼──────┼─────┤
│   5 │   10 │ 532 │
╘═════╧══════╧═════╛


## Questão 3a

## Questão 3b

In [235]:
#Trocando as linhas 2 e 3, é possível resolver pelo método de Gauss-Seigel
A = [[3, -1, 1], 
     [3, 6, 2], 
     [3, 3, 7]]

Y = [1, 0, 4]
x_init = [0, 0, 0]

In [236]:
# Check if the matrix is diagonally dominant
if check_diaDom(A):
    # Perform Gauss-Seidel method with specified tolerance
    solution = GS_method(A, Y, x_init, tol=0.001)
    print("Solution:", solution)
else:
    print("The matrix is not diagonally dominant. The Gauss-Seidel method may not converge.")


Converged after 6 iterations.
Solution: [0.0353510682848767, -0.23678862659534294, 0.6577589535616283]


## Questão 4

In [ ]:

A = array([[2.0,1.0],[5.0,7.0]])
b = array([11.0,13.0])
x = array([1.0,1.0])

sol = Jacobi(A,b, 25, x)

print("Solution for x is: ", sol)